# Dataset Scratch

In [3]:
## imports
import torch
from torch.utils.data import DataLoader  # Minibatches
import torchvision.datasets as datasets  # MNIST dataset
import torchvision.transforms as transforms
import numpy as np

Mnist Dataset

In [4]:
dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)

In [5]:
dataset

Dataset MNIST
    Number of datapoints: 60000
    Root location: dataset/
    Split: Train
    StandardTransform
Transform: ToTensor()

Taking only 0 & 1 targets from the dataset

In [6]:
exp1 = dataset.targets == 1 
exp2 = dataset.targets == 0

In [7]:
one = dataset.data[exp1]
one_t = dataset.targets[exp1]

In [8]:
zero = dataset.data[exp2]
zero_t = dataset.targets[exp2]

In [9]:
new_data = torch.cat((one, zero))
new_targets = torch.cat((one_t, zero_t))

In [10]:
## extend from torch dataset.

class CustomDataset:
    def __init__(self, data, target):
        self.data = data
        self.targets = target
        
    def __len__(self):
        return self.data
    
    def __getitem__(self, idx):
        current_sample = self.data[idx, :]
        current_target = self.targets[idx]
        return {
            "sample":torch.tensor(current_sample, dtype=torch.float),
            "target": torch.tensor(current_target, dtype=torch.long)
        }

In [11]:
new_dataset = CustomDataset(new_data, new_targets)

In [46]:
list = []
targets_list = dataset.targets.tolist()
for i in range(len(dataset)):
    if (targets_list[i] in [0, 1]):
        list.append(i)

In [47]:
list

[1,
 3,
 6,
 8,
 14,
 21,
 23,
 24,
 34,
 37,
 40,
 51,
 56,
 59,
 63,
 67,
 68,
 69,
 70,
 72,
 75,
 77,
 78,
 81,
 88,
 95,
 99,
 102,
 104,
 105,
 108,
 112,
 113,
 114,
 118,
 119,
 121,
 124,
 128,
 134,
 152,
 156,
 169,
 174,
 177,
 184,
 192,
 200,
 201,
 205,
 206,
 208,
 209,
 210,
 211,
 216,
 224,
 229,
 231,
 232,
 234,
 246,
 248,
 249,
 251,
 260,
 269,
 270,
 276,
 283,
 290,
 293,
 296,
 303,
 309,
 310,
 315,
 320,
 326,
 345,
 351,
 355,
 357,
 358,
 359,
 366,
 382,
 394,
 397,
 398,
 399,
 406,
 408,
 416,
 427,
 429,
 435,
 440,
 443,
 447,
 450,
 451,
 453,
 454,
 455,
 458,
 462,
 464,
 466,
 470,
 473,
 475,
 484,
 489,
 491,
 492,
 507,
 508,
 510,
 519,
 524,
 526,
 527,
 533,
 535,
 538,
 542,
 552,
 553,
 556,
 572,
 573,
 577,
 582,
 587,
 593,
 596,
 603,
 604,
 609,
 612,
 618,
 633,
 637,
 638,
 639,
 648,
 656,
 662,
 666,
 667,
 668,
 669,
 671,
 676,
 678,
 689,
 691,
 698,
 702,
 709,
 710,
 711,
 712,
 733,
 738,
 743,
 745,
 747,
 765,
 776,
 779,

In [48]:
trainset = torch.utils.data.Subset(dataset, list)

In [50]:
loader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=False)

In [51]:
loader

NameError: name 'laoder' is not defined

evens = list(range(0, len(trainset), 2))
odds = list(range(1, len(trainset), 2))
trainset_1 = torch.utils.data.Subset(trainset, evens)
trainset_2 = torch.utils.data.Subset(trainset, odds)

trainloader_1 = torch.utils.data.DataLoader(trainset_1, batch_size=4,
                                            shuffle=True, num_workers=2)
trainloader_2 = torch.utils.data.DataLoader(trainset_2, batch_size=4,
                                            shuffle=True, num_workers=2)